In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json
import face_recognition


2022-07-04 11:13:11.116636: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 11:13:11.116666: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#webcam_video_stream = cv2.VideoCapture(0)      #argument 0 means that we have to start default webcam of our pc
webcam_video_stream = cv2.VideoCapture('/home/ad.rapidops.com/rutvik.vasoya/Desktop/fdfdfdfdfd.mp4')    

In [3]:
#face expression model initilization
face_exp_model = model_from_json(open('facial_expression_model_structure.json','r').read())

#load the weights
face_exp_model.load_weights('facial_expression_model_weights.h5')

#list the emotion of lables
emotions_label = ('angry','disgust','fear','happy','sad','surprise','neutral')



#initialize the array variable to hold all face location in the frame
all_face_location = []



#loop through every frame in this video

while True:
     
        #get the current frame from the video stream as an image
        ret, current_frame = webcam_video_stream.read()     #this function return two value, ret is boolean if any face detect than true otherwise false and current frame from a video
        
        #resize the current frame by 1/4 size to process faster
        current_small_frame = cv2.resize(current_frame,(0,0),fx=0.25,fy=0.25)
        
        
        #now detect the all faces 
        #argumenets are image,no_of_times_to_upscale,model
        all_face_location = face_recognition.face_locations(current_small_frame,number_of_times_to_upsample=2,model='hog')
        
        
        for index,current_face_location in enumerate(all_face_location):
            top_pos,right_pos,bottom_pos,left_pos = current_face_location

#now change the position magnitute to fit the actual size video frame
            top_pos = top_pos*4
            right_pos = right_pos*4
            bottom_pos = bottom_pos*4
            left_pos = left_pos*4
            
            
            
            #extract the face from the frame and blur it , paste it back to the frame  
            current_face_image = current_frame[top_pos:bottom_pos,left_pos:right_pos]   #extract the current face
                                                                                       #from the frame
            
            
 #################################################################################################################           
            
            
            #now perprocessing image
            
            #coverted into grey scale
            current_face_image = cv2.cvtColor(current_face_image,cv2.COLOR_BGR2GRAY)
            
            
            #resize to 48X48 pixel
            current_face_image = cv2.resize(current_face_image,(48,48))
            
            
            #convert image into numpy array
            img_pixels = image.img_to_array(current_face_image)

            #expand the shape of an array into single row multiple column
            img_pixels = np.expand_dims(img_pixels,axis=0)
            
            #converted into a same sclae
            img_pixels /= 255
            
            
            #we completed preprocessing part and now we have to go to predicting facial expression
            
            exp_prediction = face_exp_model(img_pixels)
            
            #find max indexed prediction value(0 till 7)
            max_index = np.argmax(exp_prediction[0])
            
            #get corrosponding label from emotional_label
            emotion_label = emotions_label[max_index]
            
            
            #display the name as text into in the image
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(current_frame,emotion_label,(left_pos,bottom_pos),font,0.5,(255,255,255),1)
            #parameters -> image      ,text         , position,               sclae, color of text, width
          
        
  ###########################################################################################################
      
        
            cv2.rectangle(current_frame,(left_pos,top_pos),(right_pos,bottom_pos),(0,0,255),2)
                                                                                         #(r,g,b) (width of line)
            cv2.imshow("webcam  video",current_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

                
webcam_video_stream.release()
cv2.destroyAllWindows()

2022-07-04 11:13:13.424431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ad.rapidops.com/rutvik.vasoya/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-07-04 11:13:13.424450: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-04 11:13:13.424463: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (RSPL-412): /proc/driver/nvidia/version does not exist
2022-07-04 11:13:13.424604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
QObje

error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
_